# Look for redirections of german occupations (neutral forms)

In [26]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)


words=load_simple_json('de/neutral.json')
print len(words)
neutral=[]
words_all=load_simple_json('de/occupation_all.json')
words_revert={words_all[i][1]:i for i in words_all}#feminine:masculine
for i in words:
    if i not in words_all:
        if i not in words_revert:
            neutral.append(i)
        else:
            print "==F:",i
    else:
        print "===M:",i
        
print len(neutral)  

with open('de/neutral_new.json', 'w') as f:
    json.dump(neutral,  f,  indent=4)

298
===M: Fotodesigner
===M: Design-Ingenieur
===M: Schmuckdesigner
===M: Feinwerktechniker
===M: Schmuckgestalter
===M: Edelsteingestalter
===M: Maskenbildner
===M: Sozialgerontologe
===M: Altbauerneuerungstechniker
===M: Bewährungshelfer
===M: Sozialjurist
===M: Bauvermessungstechniker
===M: Landbautechniker
===M: Textildesigner
===M: Sozialpädagoge
===M: Bergbautechniker
===M: Hüttentechniker
===M: Baudenkmalpfleger
===M: Korrosionsschutztechniker
==F: Ingenieurin
===M: Abwassertechniker
===M: Elektrotechniker
276


In [27]:
from bs4 import BeautifulSoup
from wikitools import wiki, api
import json
import re


def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

    
def check_revision(revisions):
    redir=""
    page=False
    for j in revisions:
        redir_marker=["#WEITERLEITUNG","#REDIRECT","#redirect","#weiterleitung","#Weiterleitung","#Redirect"]
        if ("#WEITERLEITUNG" in j["*"])|("#REDIRECT" in j["*"])|("#redirect" in j["*"])|("#weiterleitung" in j["*"])\
        |("#Redirect" in j["*"])|("#Weiterleitung" in j["*"]):
            m = re.search(r"\[\[(.+)\]\]", j["*"])
            s = m.group(0)[2:].split('#', 1)[0].replace("]]",'')
            redir=s
        else:
            page=True
    return redir,page


def check_with_mark_beruf(beruf_m):
    beruf_markers=[" Beruf"," (Beruf)"," beruf", " (Amt)", " Amt"]
    for i in beruf_markers:
        l="de" 
        site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
        params={'action':'query','titles':beruf_m+i,'prop':'revisions','rvprop':'timestamp|content','format':'json','continue':''}
        request = api.APIRequest(site, params)
        result=request.query()
        if result["query"]["pages"].values()[0].get("revisions"):
            redir,page=check_revision(result["query"]["pages"].values()[0]["revisions"])
            if page:
                page=beruf_m+i
            return redir,page     
    return False,False
    
words=load_simple_json('de/neutral_new.json')
#words = {"Lehrer":[[],"Lehrerin"],"Professor":[[],"Professorin"], "Profesghhsor":[[],"Professorin"]}


redirection_m={}
redirection_f={}
page_m=[]
no_page_m=[]
no_page_m_new=[]
page_f=[]
no_page_f=[]
for beruf_m in words:
    l="de" 
    site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
    #last revision
    #http://de.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=timestamp|content&titles=Professorin&format=json&continue=
    
    #print beruf_m
    params={'action':'query','titles':beruf_m,'prop':'revisions','rvprop':'timestamp|content','format':'json','continue':''}
    request = api.APIRequest(site, params)
    result=request.query()
    
    if result["query"]["pages"].values()[0].get("revisions"):
        redir,page=check_revision(result["query"]["pages"].values()[0]["revisions"])
        if redir:
            redirection_m[beruf_m]=redir
        if page:
            page_m.append(beruf_m)        
    else:
        #there is no page
        #print "no page",beruf_m
        '''
        redir,page=check_with_mark_beruf(beruf_m)
        if redir:
            redirection_m[beruf_m]=redir
        elif page:
            page_m.append(page)   ######we addd beruf_m+i, where i is some substring,e.g. " (Beruf)"           
        else:
            no_page_m_new.append(beruf_m)'''
        no_page_m.append(beruf_m) 

                

In [31]:
print len(redirection_m)
print len(page_m)
print len(no_page_m)
print page_m
print redirection_m
#print len(no_page_m_new)


with open('de/wiki/n_redirects.json', 'w') as f:
    json.dump(redirection_m,  f,  indent=4)
with open('de/wiki/n_page.json', 'w') as f:
    json.dump(page_m,  f,  indent=4)
with open('de/wiki/n_no_page.json', 'w') as f:
    json.dump(no_page_m,  f,  indent=4)

14
29
233
[u'Bestattungsfachkraft', u'Kassenaufsicht', u'Feldwebel', u'Rechtsbeistand', u'Au-pair', u'Portier', u'Aufsichtsperson', u'Knappe', u'Saucier', u'Tiefbau', u'Entremetier', u'CNC-Fachkraft', u'Elektroger\xe4tefachkraft', u'Fachkraft', u'B\xfcrokraft', u'Elektrofachkraft', u'First Officer', u'Gardemanger', u'Underwriter', u'Kaltmamsell', u'Diakonisse', u'Direktrice', u'Jockey', u'Master of Business Administration', u'Produktionsfachkraft Chemie', u'Rechtswissenschaft', u'Nachtportier', u'Imam', u'Page']
{u'Spinnerei': u'Spinnen', u'Diskjockey': u'DJ', u'Regeltechniker': u'Prozessleitelektroniker', u'Verpackungstechniker': u'Verpackungsmittelmechaniker', u'Demichef de rang': u'Kellner', u'Chemieverfahrenstechniker': u'Chemikant', u'Autosattlerei': u'Autosattler', u'Human Resources': u'Personalwesen', u'IT-Consultant': u'IT-Berater', u'Schiffsjunge': u'Seemann', u'Comedian': u'Komiker', u'Chef de rang': u'Chef de Rang', u'Fotomodell': u'Model', u'Lehrkraft': u'Lehrer'}


In [3]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
redirection_n=load_simple_json('de/wiki/n_redirects.json')
words=load_simple_json('de/occupation_all.json')
words_revert={words[i][1]:i for i in words}#feminine:masculine

n_links_to_masculine={}
n_links_to_feminine={}
n_links_to_smth={}
for i in redirection_n:
    if words.has_key(redirection_n[i]):
        n_links_to_masculine[i]=redirection_n[i]
    elif words_revert.has_key(redirection_n[i]):
        n_links_to_feminine[i]=redirection_n[i]
    else:
        #still links to masculine?
        #if redirection_n[i][-2:]=="er":
        #redirects to masculine
        n_links_to_smth[i]=redirection_n[i]
        
'''        
with open('de/wiki/f_links_to_mascuilne.json', 'w') as f:
    json.dump(f_links_to_mascuilne,  f,  indent=4)
with open('de/wiki/f_links_to_smth.json', 'w') as f:
    json.dump(f_links_to_smth,  f,  indent=4)'''

"        \nwith open('de/wiki/f_links_to_mascuilne.json', 'w') as f:\n    json.dump(f_links_to_mascuilne,  f,  indent=4)\nwith open('de/wiki/f_links_to_smth.json', 'w') as f:\n    json.dump(f_links_to_smth,  f,  indent=4)"

In [8]:
print n_links_to_masculine
print n_links_to_feminine
print n_links_to_smth

{u'Regeltechniker': u'Prozessleitelektroniker', u'Demichef de rang': u'Kellner', u'Chemieverfahrenstechniker': u'Chemikant', u'IT-Consultant': u'IT-Berater', u'Comedian': u'Komiker', u'Verpackungstechniker': u'Verpackungsmittelmechaniker', u'Lehrkraft': u'Lehrer'}
{}
{u'Spinnerei': u'Spinnen', u'Diskjockey': u'DJ', u'Autosattlerei': u'Autosattler', u'Human Resources': u'Personalwesen', u'Schiffsjunge': u'Seemann', u'Chef de rang': u'Chef de Rang', u'Fotomodell': u'Model'}


# Check if existing page in one of subcategories Beruf

In [32]:
from wikitools import wiki, api,page, category
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
no_page_m=load_simple_json('de/wiki/m_no_page.json')
no_page_f=load_simple_json('de/wiki/f_no_page.json')
page_m=load_simple_json('de/wiki/m_page.json')
page_f=load_simple_json('de/wiki/f_page.json')

m_links_to_feminine=load_simple_json('de/wiki/m_links_to_feminine.json')
m_links_to_smth=load_simple_json('de/wiki/m_links_to_smth.json')
f_links_to_mascuilne=load_simple_json('de/wiki/f_links_to_mascuilne.json')
f_links_to_smth=load_simple_json('de/wiki/f_links_to_smth.json')
words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
words_revert={words[i][1]:i for i in words}#feminine:masculine
beruf_subcategories=load_simple_json('de/wiki/beruf_subcategories.json')

def check_categories(page,categories):
    #http://de.wikipedia.org/w/api.php?action=query&titles=Hebamme&prop=categories
    l="de" 
    site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
    params={'action':'query','titles':page,'prop':'categories','format':'json','continue':''}
    request = api.APIRequest(site, params)
    result=request.query()  
    cat=[]
    if result["query"]["pages"].values()[0].get("categories"):
        for i in result["query"]["pages"].values()[0]["categories"]:
            cat.append(i["title"])
            if i["title"] in categories:
                #print i ["title"]
                return True  
        return False
    else:
        print "no Categories here"
        return False
    


####
def check_categories_substring(page,overloop=0,seen_cat=[]): #firstly go here
    global beruf_subcategories
    where_go_categories_=[]
    l="de"
    seen_cat.append(page)
    #print "Firstly check categories of this page: "+ page
    #http://de.wikipedia.org/w/api.php?action=query&titles=Hebamme&prop=categories
    site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
    params={'action':'query','titles':page,'prop':'categories','format':'json','continue':''}
    request = api.APIRequest(site, params)
    result=request.query()
    if result["query"]["pages"].values()[0].get("categories"):
        for i in result["query"]["pages"].values()[0]["categories"]:
            if ("Kategorie:Wikipedia:" not in i["title"])&("Kategorie:!" not in i["title"]):
                if ("beruf" in i["title"])|("arbeit" in i["title"])|("Arbeit" in i["title"])|("amt" in i["title"])|("Amt" in i["title"])|("Beruf"in i["title"])|("wissenschaftler"in i["title"])|("Wissenschaftler"in i["title"]):#| ("Personenbezeichnung" in i["title"]):
                    #print "--->",i["title"]," is word beruf"
                    return True
                elif i["title"] in beruf_subcategories:
                    #print "--->",i["title"]," in beruf sub"
                    return True  
                else:
                    if i["title"] not in seen_cat:
                        where_go_categories_.append(i["title"])                    
    for c in where_go_categories_ :
        if c not in seen_cat:
            if overloop<4:
                if check_categories_substring(c,overloop=overloop+1,seen_cat=seen_cat):
                    return True
            else:
                #print "overloop ==="+c
                pass
    return False     
       
def check_links(page,link):
    #http://de.wikipedia.org/w/api.php?action=query&titles=Hebamme&prop=categories|links
    l="de" 
    site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
    params={'action':'query','titles':page,'prop':'categories|links','format':'json','continue':''}
    request = api.APIRequest(site, params)
    result=request.query()  
    links=[]
    if result["query"]["pages"].values()[0].get("links"):
        for i in result["query"]["pages"].values()[0]["links"]:
            links.append(i["title"])
            if i["title"]==link:
                return True  
        return False
    else:
        print "no links"
        return False
    
def check_page_category(page):    
    page_validated=[]
    page_ambigious=[]
    page_other=[]
    ambigious_de=[u'Kategorie:Begriffskl\xe4rung']   

    for i in page:
        if check_categories(i,beruf_subcategories):
            #print i, "in Beruf subcategory"
            page_validated.append(i)
        else:
            #print i, "==not in Beruf subcategory"
            if check_categories(i,ambigious_de):
                #print i, "in Kategorie:Begriffsklärung"
                page_ambigious.append(i)
                if words_revert.has_key(i):
                    if check_links(i,words_revert[i]):
                        print i,"Ambigious!!! have link to maskuline",words_revert[i]
                if words.has_key(i):
                    if check_links(i,words[i]):
                        print i,"Ambigious!!! have link to feminine",words[i]
                if check_links(i,i+" (Beruf)"):
                        print "Ambigious page has link to:",i+" (Beruf)"       
                #maybe learn ambigious pages text???
                '''for j in words:
                    if check_links(i,j):
                        print "Ambigious!!! have link to another Beruf:",j
                        
                    if check_links(i,words[j][1]):
                        print "Ambigious!!! have link to another Beruf:",words[j][1]'''                      
                       
            else:
                #print "Start checking...",i 
                #check if the categories and there parent cat. has world "beruf"or "amt" or it's in beruf_subcat
                if check_categories_substring(i):
                    page_validated.append(i)
                else:
                    #print i, "====== just another page"
                    page_other.append(i)
                
    return page_validated,page_ambigious,page_other

In [33]:
#page_f_validated,page_f_ambigious,page_f_other

page_n=load_simple_json('de/wiki/n_page.json')
page_n_validated,page_n_ambigious,page_n_other=check_page_category(page_n)

Ambigious page has link to: Direktrice (Beruf)


In [34]:
print page_n_validated
print page_n_ambigious
print page_n_other
with open('de/wiki/n_page_validated.json', 'w') as f:
    json.dump(page_n_validated,  f,  indent=4)
with open('de/wiki/n_page_ambigious.json', 'w') as f:
    json.dump(page_n_ambigious,  f,  indent=4)    
with open('de/wiki/n_page_other.json', 'w') as f: #wrong page not occupation
    json.dump(page_n_other,  f,  indent=4)   

[u'Bestattungsfachkraft', u'Feldwebel', u'Rechtsbeistand', u'Au-pair', u'Portier', u'Aufsichtsperson', u'Saucier', u'Entremetier', u'CNC-Fachkraft', u'Elektroger\xe4tefachkraft', u'B\xfcrokraft', u'Elektrofachkraft', u'Gardemanger', u'Kaltmamsell', u'Diakonisse', u'Jockey', u'Produktionsfachkraft Chemie', u'Rechtswissenschaft', u'Nachtportier', u'Imam']
[u'Knappe', u'First Officer', u'Underwriter', u'Direktrice', u'Page']
[u'Kassenaufsicht', u'Tiefbau', u'Fachkraft', u'Master of Business Administration']


# Tables

In [36]:
from prettytable import PrettyTable

words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
words_revert={words[i][1]:i for i in words}#feminine:masculine
neutral=load_simple_json('de/neutral_new.json')

no_page_m=load_simple_json('de/wiki/m_no_page.json')
no_page_f=load_simple_json('de/wiki/f_no_page.json')
no_page_n=load_simple_json('de/wiki/n_no_page.json')
page_m=load_simple_json('de/wiki/m_page.json')
page_f=load_simple_json('de/wiki/f_page.json')
page_n=load_simple_json('de/wiki/n_page.json')


page_f_validated=load_simple_json('de/wiki/f_page_validated.json')
page_f_ambigious=load_simple_json('de/wiki/f_page_ambigious.json')
page_f_other=load_simple_json('de/wiki/f_page_other.json')  
    
page_m_validated=load_simple_json('de/wiki/m_page_validated.json') 
page_m_ambigious=load_simple_json('de/wiki/m_page_ambigious.json')
page_m_other=load_simple_json('de/wiki/m_page_other.json')

page_n_validated=load_simple_json('de/wiki/n_page_validated.json')
page_n_ambigious=load_simple_json('de/wiki/n_page_ambigious.json')
page_n_other=load_simple_json('de/wiki/n_page_other.json') 

m_links_to_feminine=load_simple_json('de/wiki/m_links_to_feminine.json')
m_links_to_smth=load_simple_json('de/wiki/m_links_to_smth.json')
f_links_to_mascuilne=load_simple_json('de/wiki/f_links_to_mascuilne.json')
f_links_to_smth=load_simple_json('de/wiki/f_links_to_smth.json')
redirection_n=load_simple_json('de/wiki/n_redirects.json')



t = PrettyTable(['----',"Occup.", 'Wiki pages', "Validated pages","Ambigious pages","Redirects",
                 "Redir. to oposit gender"])
t.add_row(['Masculine',len(words), len(page_m), len(page_m_validated), len(page_m_ambigious),
           len(m_links_to_feminine)+len(m_links_to_smth), len(m_links_to_feminine)])
t.add_row(['Feminine',len(words_revert), len(page_f), len(page_f_validated), len(page_f_ambigious),
           len(f_links_to_mascuilne)+len(f_links_to_smth), len(f_links_to_mascuilne)])
t.add_row(['Neutral',len(neutral), len(page_n), len(page_n_validated), len(page_n_ambigious),
           len(redirection_n), "--"])
print t

+-----------+--------+------------+-----------------+-----------------+-----------+-------------------------+
|    ----   | Occup. | Wiki pages | Validated pages | Ambigious pages | Redirects | Redir. to oposit gender |
+-----------+--------+------------+-----------------+-----------------+-----------+-------------------------+
| Masculine |  3886  |    818     |       664       |       115       |    487    |            4            |
|  Feminine |  3886  |     12     |        7        |        3        |    266    |           176           |
|  Neutral  |  276   |     29     |        20       |        5        |     14    |            --           |
+-----------+--------+------------+-----------------+-----------------+-----------+-------------------------+


In [37]:
t2 = PrettyTable(['----',"Occup.", 'Wiki pages', "Redirects", "No page"])
t2.add_row(['Masculine',len(words), len(page_m), len(m_links_to_feminine)+len(m_links_to_smth),
            len(no_page_m)])
t2.add_row(['Feminine',len(words_revert), len(page_f),len(f_links_to_mascuilne)+len(f_links_to_smth),
            len(no_page_f)])
t2.add_row(['Neutral',len(neutral), len(page_n),len(redirection_n),len(no_page_n)])
print t2
print
t3 = PrettyTable(['----','Wiki pages', "Validated pages","Ambigious pages","Not validated pages"])
t3.add_row(['Masculine', len(page_m), len(page_m_validated), len(page_m_ambigious),len(page_m_other)])
t3.add_row(['Feminine', len(page_f), len(page_f_validated), len(page_f_ambigious),len(page_f_other)])
t3.add_row(['Neutral', len(page_n), len(page_n_validated), len(page_n_ambigious),len(page_n_other)])
print t3
print
t4 = PrettyTable(['----',"Redirects","Redirects to oposit gender", "Other redirects"])
t4.add_row(['Masculine',len(m_links_to_feminine)+len(m_links_to_smth),
            len(m_links_to_feminine),len(m_links_to_smth)])
t4.add_row(['Feminine',len(f_links_to_mascuilne)+len(f_links_to_smth),
            len(f_links_to_mascuilne),len(f_links_to_smth)])
t4.add_row(['Neutral',len(redirection_n),"--","--"])
print t4


+-----------+--------+------------+-----------+---------+
|    ----   | Occup. | Wiki pages | Redirects | No page |
+-----------+--------+------------+-----------+---------+
| Masculine |  3886  |    818     |    487    |   2581  |
|  Feminine |  3886  |     12     |    266    |   3608  |
|  Neutral  |  276   |     29     |     14    |   233   |
+-----------+--------+------------+-----------+---------+

+-----------+------------+-----------------+-----------------+---------------------+
|    ----   | Wiki pages | Validated pages | Ambigious pages | Not validated pages |
+-----------+------------+-----------------+-----------------+---------------------+
| Masculine |    818     |       664       |       115       |          39         |
|  Feminine |     12     |        7        |        3        |          2          |
|  Neutral  |     29     |        20       |        5        |          4          |
+-----------+------------+-----------------+-----------------+---------------------